In [1]:
! pip install scikit-learn-intelex


In [2]:
from sklearnex import patch_sklearn
patch_sklearn()

import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

PATH="/kaggle/input/webis-clickbait-22/"
sep='[SEP]'

label_encoder = LabelEncoder()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:

def get_split_data(file_path,if_train=True):
    global label_encoder
    data=pd.read_json(file_path,lines=True,encoding='utf-8')

    y = data["tags"]
    Y = [i[0] for i in y]
    print("len of Y: ",len(Y))
    if if_train:
        Y = label_encoder.fit_transform(Y)
    else:
        Y = label_encoder.transform(Y)

    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords']]
    dic=new_data.to_dict(orient='records')

    desccount=0
    keycount=0
    for i in dic:
        # print(i)
        i['postText']='postText: '+i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']='targetParagraphs: '+targetparah
        i['targetTitle']='targetTitle: '+i['targetTitle']
        targetDescription=i['targetDescription']
        if(targetDescription is None or len(targetDescription)==0):
            targetDescription='none'
            desccount+=1
        i['targetDescription']='targetDescription: '+targetDescription
        targetkey=i['targetKeywords']
        if(targetkey is None or len(targetkey)==0):
            targetkey='none'
            keycount+=1
        i['targetKeywords']='targetKeywords: '+targetkey
        i['text']=i['postText']+sep+i['targetParagraphs']+sep+i['targetTitle']+sep+i['targetDescription']+sep+i['targetKeywords']
        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']

    print("desccount: ",desccount)
    print("keycount: ",keycount)
    X = [i['text'] for i in dic]
    X=np.array(X)
#     return X[0:100],Y[0:100]
    return X,Y


In [4]:

x_train,y_train = get_split_data(PATH+'train.jsonl')
x_test,y_test = get_split_data(PATH+'test.jsonl',False)
x_val,y_val = get_split_data(PATH+'validation.jsonl',False)


len of Y:  3200
desccount:  332
keycount:  1314
len of Y:  1000
desccount:  104
keycount:  414
len of Y:  800
desccount:  78
keycount:  330


In [5]:
# len(x_train),len(y_train),len(x_test),len(y_test),len(x_val),len(y_val),len(x_train[0]),len(y_train[0])
x_train.shape,y_train.shape,x_test.shape,y_test.shape,x_val.shape,y_val.shape,len(x_train[10])


((3200,), (3200,), (1000,), (1000,), (800,), (800,), 3518)

In [6]:
type(x_train),type(y_train),type(x_test),type(y_test),type(x_val),type(y_val),type(x_train[0]),type(y_train[0])

(numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.str_,
 numpy.int64)

In [7]:
vectorizer = TfidfVectorizer(ngram_range=(1,1),stop_words=None)
x_train =  vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)
x_val = vectorizer.transform(x_val)

INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


In [8]:
param_grid = {
    'model_xgb__learning_rate': [0.01,0.1],
    'model_xgb__n_estimators': [100, 500, 1000],
    'model_xgb__max_depth': [3, 5, 7]
    }

#     ('to_coo', FunctionTransformer(lambda x: x.tocoo()),
#     ('to_tensor',FunctionTransformer(lambda x: csr_to_sparse_tensor(x))),
xgb_model=XGBClassifier(n_jobs=-1,learning_rate=0.1,n_estimators=1000,max_depth=7)
# xgb_model=XGBClassifier(n_jobs=-1)
xgb_model.fit(x_train,y_train)
# print(tf_idf_pipeline.fit(x_train,y_train))
# print(tf_idf_pipeline.predict(x_train))
# grid_search = GridSearchCV(tf_idf_pipeline, param_grid, cv=5, n_jobs=-1,verbose=10)
# grid_search.fit(x_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=-1,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [9]:
# optimal_rate = grid_search.best_params_["learning_rate"]
# optimal_estimators = grid_search.best_params_["n_estimators"]
# optimal_depth=grid_search.best_params_["max_depth"]
# optimal_ngram = grid_search.best_params_["vectorizer__ngram_range"]
# optimal_stop_words = grid_search.best_params_["vectorizer__stop_words"]
# print("optimal_rate: ",optimal_rate)
# print("optimal_estimators: ",optimal_estimators)
# print("optimal_depth: ",optimal_depth)
# print("optimal_ngram: ",optimal_ngram)
# print("optimal_stop_words: ",optimal_stop_words)
# # print(grid_search.best_estimator_)
# print("Best parameters found by GridSearchCV:")
# print(grid_search.best_params_)



In [10]:

# vectorizer = TfidfVectorizer(ngram_range=optimal_ngram,stop_words=optimal_stop_words)
# x_train =  vectorizer.fit_transform(x_train)
# x_test = vectorizer.transform(x_test)
# x_val = vectorizer.transform(x_val)
# print(x_train.shape)
# print(x_test.shape)



In [11]:
# xgb_model=XGBClassifier(learning_rate=optimal_rate,n_estimators=optimal_estimators,max_depth=optimal_depth,n_jobs=-1)
# xgb_model.fit(X_train, y_train)
y_train_pred = xgb_model.predict(x_train)
y_test_pred = xgb_model.predict(x_test)
y_val_pred = xgb_model.predict(x_val)
print("Training data metrics : ")
print(accuracy_score(y_train_pred,y_train))

print("Testing data accuracy")
print(accuracy_score(y_test,y_test_pred))

print("Validation data accuracy")
print(accuracy_score(y_val,y_val_pred))



Training data metrics : 
1.0
Testing data accuracy
0.547
Validation data accuracy
0.5325


In [12]:
print("Training data metrics:")
print(balanced_accuracy_score(y_train_pred, y_train))
print(f1_score(y_train_pred, y_train, average='weighted'))
print(classification_report(y_train_pred, y_train))

print("Testing data metrics:")
print(balanced_accuracy_score(y_test_pred, y_test))
print(f1_score(y_test_pred, y_test, average='weighted'))
print(classification_report(y_test_pred, y_test))

print("Validation data metrics:")
print(balanced_accuracy_score(y_val_pred, y_val))
print(f1_score(y_val_pred, y_val, average='weighted'))
print(classification_report(y_val_pred, y_val))

Training data metrics:
1.0
1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       559
           1       1.00      1.00      1.00      1274
           2       1.00      1.00      1.00      1367

    accuracy                           1.00      3200
   macro avg       1.00      1.00      1.00      3200
weighted avg       1.00      1.00      1.00      3200

Testing data metrics:
0.56007526492108
0.5551968279038493
              precision    recall  f1-score   support

           0       0.29      0.60      0.39        84
           1       0.60      0.53      0.56       454
           2       0.60      0.55      0.58       462

    accuracy                           0.55      1000
   macro avg       0.50      0.56      0.51      1000
weighted avg       0.58      0.55      0.56      1000

Validation data metrics:
0.523480550641534
0.5425220888735143
              precision    recall  f1-score   support

           0       0.26      0.5

In [13]:
pickle.dump((xgb_model,label_encoder,vectorizer),open('model_xgb.pkl','wb'))

In [14]:
model2,encoder,vectorizer = pickle.load(open("model_xgb.pkl","rb"))

In [15]:
y_train_pred = model2.predict(x_train)
y_test_pred = model2.predict(x_test)
y_val_pred = model2.predict(x_val)
print("Training data metrics : ")
print(accuracy_score(y_train_pred,y_train))

print("Testing data accuracy")
print(accuracy_score(y_test,y_test_pred))

print("Testing data accuracy")
print(accuracy_score(y_val,y_val_pred))

Training data metrics : 
1.0
Testing data accuracy
0.547
Testing data accuracy
0.5325


In [16]:
print("Training data metrics:")
print(balanced_accuracy_score(y_train_pred, y_train))
print(f1_score(y_train_pred, y_train, average='weighted'))
print(classification_report(y_train_pred, y_train))

print("Testing data metrics:")
print(balanced_accuracy_score(y_test_pred, y_test))
print(f1_score(y_test_pred, y_test, average='weighted'))
print(classification_report(y_test_pred, y_test))

print("Validation data metrics:")
print(balanced_accuracy_score(y_val_pred, y_val))
print(f1_score(y_val_pred, y_val, average='weighted'))
print(classification_report(y_val_pred, y_val))

Training data metrics:
1.0
1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       559
           1       1.00      1.00      1.00      1274
           2       1.00      1.00      1.00      1367

    accuracy                           1.00      3200
   macro avg       1.00      1.00      1.00      3200
weighted avg       1.00      1.00      1.00      3200

Testing data metrics:
0.56007526492108
0.5551968279038493
              precision    recall  f1-score   support

           0       0.29      0.60      0.39        84
           1       0.60      0.53      0.56       454
           2       0.60      0.55      0.58       462

    accuracy                           0.55      1000
   macro avg       0.50      0.56      0.51      1000
weighted avg       0.58      0.55      0.56      1000

Validation data metrics:
0.523480550641534
0.5425220888735143
              precision    recall  f1-score   support

           0       0.26      0.5